In [2]:
using IntersectionTheory

This notebook covers the examples from *Schubert2* [documentations](https://faculty.math.illinois.edu/Macaulay2/doc/Macaulay2/share/doc/Macaulay2/Schubert2/html/) and performs the computations in *IntersectionTheory*.
### Lines on hypersurfaces
The famous 27 lines on a cubic surface.

In [3]:
G = grassmannian(2, 4)
S, Q = G.bundles
B = symmetric_power(3, dual(S))
c = chern(B.rank, B) # can also use ctop(B)
c, integral(c)

(27*c_{2}^2, 27)

In [4]:
f = n -> (G = grassmannian(2, n+1);
    (S, Q) = G.bundles;
    symmetric_power(2n-3, dual(S)) |> ctop |> integral)
[@time f(n) for n in 2:10]

  0.049681 seconds (47.02 k allocations: 2.859 MiB, 97.99% compilation time)
  0.001237 seconds (4.38 k allocations: 128.445 KiB)
  0.001722 seconds (11.21 k allocations: 247.250 KiB)
  0.002474 seconds (15.22 k allocations: 346.430 KiB)
  0.003705 seconds (28.71 k allocations: 563.898 KiB)
  0.005586 seconds (46.63 k allocations: 843.391 KiB)
  0.008250 seconds (77.31 k allocations: 1.365 MiB)
  0.011781 seconds (108.55 k allocations: 1.736 MiB)
  0.019889 seconds (158.72 k allocations: 2.460 MiB)


9-element Vector{Nemo.fmpq}:
 1
 27
 2875
 698005
 305093061
 210480374951
 210776836330775
 289139638632755625
 520764738758073845321

In general, we can consider $k$-planes contained in a degree-$d$ hypersurface in $\mathbf P^n$; when $n = \frac1{k+1}\binom{d+k}{d}+k$, we get a finite number.

For larger $k$ and $d$, using Bott's formula to evaluate the integral is more efficient.

To do this, use `grassmannian(k, n, bott=true)` to create a version of $\mathrm{Gr}(k,n)$ with a $T^n$-action. Most of the syntax are the same when working with equivariant bundles:
- `G.bundles` returns the subbundle and the quotient bundle;
- `G.T` is the tangent bundle;
- the operators `symmetric_power`, `exterior_power`, `dual`, `det`, direct sum, tensoring work the same.

In [5]:
g = (k, d; bott=false) -> (
    n = Int(binomial(d+k, d) // (k+1)) + k; @show n;
    G = grassmannian(k+1, n+1, bott=bott);
    (S, Q) = G.bundles;
    symmetric_power(d, dual(S)) |> ctop |> integral)
@time g(3, 5)

n = 17
 15.035461 seconds (63.58 M allocations: 1.453 GiB, 0.46% gc time, 2.42% compilation time)


64127725294951805931404297113125

In [7]:
@time g(3, 5, bott=true)

n = 17
  0.461386 seconds (4.04 M allocations: 202.080 MiB, 25.18% gc time)


64127725294951805931404297113125

### Conics on a quintic threefold

In [8]:
G = grassmannian(3, 5)
S, Q = G.bundles
B = symmetric_power(2, dual(S))
X = proj(B)
A = symmetric_power(5, dual(S)) - symmetric_power(3, dual(S)) * X.bundles[1]
integral(chern(A))

609250

### The Horrocks-Mumford bundle
To introduce parameters, we use the `param` argument.

In [9]:
X, n = proj(4, param = "n")
h = X.O1
F = bundle(X, 2, 1 + 5h + 10h^2)

AbsBundle of rank 2 on AbsVariety of dim 4

In [10]:
G = 2*exterior_power(2, cotangent_bundle(X)) * OO(X, 2) - 5*OO(X, -1) - 5*OO(X)
chern(G)

1 - h + 4*h^2

In [11]:
F == dual(G) * OO(X, 2)

true

In [12]:
chi(F * OO(X, n)) # this returns a number in a function field

(n^4 + 20*n^3 + 125*n^2 + 250*n + 24)//12

In [13]:
hilbert_polynomial(F) # this is a true polynomial

1//12*t^4 + 5//3*t^3 + 125//12*t^2 + 125//6*t + 2

### Riemann-Roch on a curve
As before, we use `FunctionField` to introduce the two extra parameters.

In [14]:
X, (e,f,D,K), (r,s) = variety(1, ["e","f","D","K"], [1,1,1,1], param = ["r","s"])
X.T = bundle(X, 1, 1-K)
chi(OO(X)), chi(OO(X, D))

(-1//2*K, D - 1//2*K)

In [15]:
E, F = bundle(X, r, 1+e), bundle(X, s, 1+f)

(AbsBundle of rank r on AbsVariety of dim 1, AbsBundle of rank s on AbsVariety of dim 1)

In [16]:
chi(hom(E, F))

-s*e + r*f - (r*s)//2*K

### Riemann-Roch on a surface

In [17]:
X, (D,d1,K,c2,d2), r = variety(2, ["D","d_1","K","c_2","d_2"], [1,1,1,2,2], param = "r")
X.T = bundle(X, 2, 1-K+c2)
todd(X), chi(OO(X))

(1 - 1//2*K + 1//12*K^2 + 1//12*c_2, 1//12*K^2 + 1//12*c_2)

In [18]:
E = bundle(X, r, 1+d1+d2)

AbsBundle of rank r on AbsVariety of dim 2

In [19]:
chi(E - rank(E) * OO(X)), chi(OO(X, D) - OO(X))

(1//2*d_1^2 - 1//2*d_1*K - d_2, 1//2*D^2 - 1//2*D*K)

In [20]:
chi(OO(section_zero_locus(OO(X, D)))) # here D and K are the pullback to D

-1//2*D - 1//2*K

In [21]:
p_a = D -> 1 - chi(OO(D))
P, n = proj(2, param = "n")
p_a(section_zero_locus(OO(P, n))) # arithmetic genus of a plane curve of degree n; this is a number in Fn

(n^2 - 3*n + 2)//2

In [22]:
P1, (m,n) = proj(1, param = ["m", "n"]); P2 = proj(1, base=P1.base)
PxP = P1 * P2
h = pullback(PxP→P1, P1.O1)
k = pullback(PxP→P2, P2.O1)
p_a(section_zero_locus(OO(PxP, m*h+n*k)))

m*n - m - n + 1

### Riemann-Roch without denominators

In [23]:
f = (n,d,e) -> (
    (X, (D, E)) = variety(n, [d=>"d", e=>"e"]);
    p = chern(exterior_power(dual(D)) * E) - 1;
    p ÷ ctop(D))
f(4,1,1), f(4,1,2), f(4,1,3), f(4,1,4)

(1 + d_{1} - e_{1} + d_{1}^2 - 2*d_{1}*e_{1} + e_{1}^2 + d_{1}^3 - 3*d_{1}^2*e_{1} + 3*d_{1}*e_{1}^2 - e_{1}^3, 2 + 3*d_{1} - e_{1} + 4*d_{1}^2 - 3*d_{1}*e_{1} + e_{1}^2 - 2*e_{2} + 5*d_{1}^3 - 6*d_{1}^2*e_{1} + 4*d_{1}*e_{1}^2 - e_{1}^3 - 7*d_{1}*e_{2} + 3*e_{1}*e_{2}, 3 + 6*d_{1} - e_{1} + 10*d_{1}^2 - 4*d_{1}*e_{1} + e_{1}^2 - 2*e_{2} + 15*d_{1}^3 - 10*d_{1}^2*e_{1} + 5*d_{1}*e_{1}^2 - e_{1}^3 - 9*d_{1}*e_{2} + 3*e_{1}*e_{2} - 3*e_{3}, 4 + 10*d_{1} - e_{1} + 20*d_{1}^2 - 5*d_{1}*e_{1} + e_{1}^2 - 2*e_{2} + 35*d_{1}^3 - 15*d_{1}^2*e_{1} + 6*d_{1}*e_{1}^2 - e_{1}^3 - 11*d_{1}*e_{2} + 3*e_{1}*e_{2} - 3*e_{3})

### A cubic fourfold containing a degree-5 del Pezzo surface
The *Schubert2* documentation on this one is quite involved. In fact, the construction of the del Pezzo surface is just a series of blowups.

In [24]:
P2, (r,s) = proj(2, param = ["r", "s"])
S = blowup_points(4, P2)
K = canonical_class(S)

e_{4} + e_{3} + e_{2} + e_{1} - 3*h

In [25]:
Y = complete_intersection(proj(5, base=S.base), 3)
# here it's important to use `inclusion=true` since usually there is no algebraic class in Y for S
i = hom(S, Y, [-K], inclusion=true)
Bl, E = blowup(i)

(AbsVariety of dim 4, AbsVariety of dim 3)

In [26]:
h = (Bl → Y).pullback(Y.O1)
e = (E → Bl).pushforward(E(1))
chi(bundle(Bl, 1, 2h-e)), integral((2h-e)^4)

(5, 1)

In [27]:
chi(bundle(Bl, 1, r*h+s*e)), integral((r*h+s*e)^4)

((3*r^4 - 30*r^2*s^2 - 40*r*s^3 - 7*s^4 + 18*r^3 + 30*r^2*s - 30*r*s^2 - 46*s^3 + 45*r^2 + 70*r*s - 5*s^2 + 54*r + 58*s + 24)//24, 3*r^4 - 30*r^2*s^2 - 40*r*s^3 - 7*s^4)

Moreover, we can compute the intersection matrix of the cubic and verify that it has determinant 14 (to be honest, the only missing ingredient is the self-intersection number of $S$ in $Y$, which can be computed as the integral of the top Chern class of the normal bundle, and does not need the above construction).

In [28]:
Y1 = i.codomain
h = Y1.O1
s = pushforward(i, S(1))
M = intersection_matrix([h^2, s])
M, det(M), integral(ctop(-i.T)) # the last one is the top Chern class of the normal bundle, which indeed gives 13

([3 5; 5 13], 14, 13)

### Examples from Schubert
#### Hilbert polynomial and Todd class of projective 3-space

In [29]:
P, n = proj(3, param = "n")
chi(OO(P, n))

(n^3 + 6*n^2 + 11*n + 6)//6

In [30]:
Nemo.factor(hilbert_polynomial(proj(3)))

(1//6) * (t + 1) * (t + 3) * (t + 2)

In [31]:
todd(proj(3))

1 + 2*h + 11//6*h^2 + h^3

#### Generation of formulas

In [32]:
X, (A,B) = variety(4, [2=>"c", 3=>"d"])
chern(A)

1 + c_{1} + c_{2}

In [33]:
segre(B)

1 - d_{1} + d_{1}^2 - d_{2} - d_{1}^3 + 2*d_{1}*d_{2} - d_{3} + d_{1}^4 - 3*d_{1}^2*d_{2} + d_{2}^2 + 2*d_{1}*d_{3}

In [34]:
chern(A*B)

1 + 3*c_{1} + 2*d_{1} + 3*c_{1}^2 + 5*c_{1}*d_{1} + d_{1}^2 + 3*c_{2} + 2*d_{2} + c_{1}^3 + 4*c_{1}^2*d_{1} + 2*c_{1}*d_{1}^2 + 6*c_{1}*c_{2} + 4*c_{2}*d_{1} + 4*c_{1}*d_{2} + 2*d_{1}*d_{2} + 2*d_{3} + c_{1}^3*d_{1} + c_{1}^2*d_{1}^2 + 3*c_{1}^2*c_{2} + 6*c_{1}*c_{2}*d_{1} + 2*c_{2}*d_{1}^2 + 3*c_{1}^2*d_{2} + 3*c_{1}*d_{1}*d_{2} + 3*c_{2}^2 + d_{2}^2 + 3*c_{1}*d_{3} + 2*d_{1}*d_{3}

In [35]:
chern(3, symmetric_power(3, dual(A)))

-6*c_{1}^3 - 30*c_{1}*c_{2}

In [36]:
segre(2, hom(exterior_power(2, A), exterior_power(2, B)))

6*c_{1}^2 - 8*c_{1}*d_{1} + 3*d_{1}^2 - d_{2}

#### Grassmannian of lines in $\mathbf P^3$

In [37]:
G, n = grassmannian(2, 4, param = "n")
Q = G.bundles[2]
chi(symmetric_power(n, Q))

(n^3 + 6*n^2 + 11*n + 6)//6

In [38]:
chi(OO(G, n)), hilbert_polynomial(G)

((n^4 + 8*n^3 + 23*n^2 + 28*n + 12)//12, 1//12*t^4 + 2//3*t^3 + 23//12*t^2 + 7//3*t + 1)

In [39]:
P5 = proj(5, base=G.base)
chi(OO(P5, n) - OO(P5, n-2)), hilbert_polynomial(complete_intersection(P5, 2))

((n^4 + 8*n^3 + 23*n^2 + 28*n + 12)//12, 1//12*t^4 + 2//3*t^3 + 23//12*t^2 + 7//3*t + 1)

#### Lines and conics on a quintic threefold

In [40]:
G = grassmannian(2, 5)
S, Q = G.bundles
B = symmetric_power(5, dual(S))
ctop(B), integral(ctop(B))

(2875*c_{2}^3, 2875)

In [41]:
G = grassmannian(3, 5)
S, Q = G.bundles
B = symmetric_power(2, dual(S))
X = proj(B)
A = symmetric_power(5, dual(S)) - symmetric_power(3, dual(S)) * X.bundles[1]
ctop(A), pushforward(X → G, ctop(A)), integral(ctop(A))

(609250*h^5*c_{3}^2, 609250*c_{3}^2, 609250)

#### Count the number of space conics intersecting 8 given lines

In [42]:
G = grassmannian(3, 4)
S, Q = G.bundles
F = proj(symmetric_power(2, dual(S)))
d = pullback(F → G, G.O1)
e = F.O1
integral((2d+e)^8)

92

#### Euler characteristic of Horrocks-Mumford bundle
Already done above.
#### Riemann-Roch formulas

In [43]:
X, (K,c2,c3,D) = variety(3, ["K", "c_2", "c_3", "D"], [1,2,3,1])
X.T = bundle(X, 3, 1-K+c2+c3)
chi(OO(X, D))

1//12*K^2*D - 1//4*K*D^2 + 1//6*D^3 - 1//24*K*c_2 + 1//12*c_2*D

#### The number of elliptic cubics on a sextic 4-fold

In [44]:
G = grassmannian(3, 6)
S, Q = G.bundles
B = symmetric_power(3, dual(S))
X = proj(B)
A = symmetric_power(6, dual(S)) - symmetric_power(3, dual(S)) * X.bundles[1]
integral(ctop(A))

2734099200